In [31]:
import numpy as np

In [39]:
import matplotlib.pyplot as plt
from matplotlib import rc

plt.rcParams["figure.figsize"] = [12, 12]
# If you have problems with latex at matplotlib just comment next two lines, this might help
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
 

def fix_scaling(ax=None):
    if not ax:
        xlim = plt.xlim()
        ylim = plt.ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            plt.ylim((ylim[0] - (d1 - d2) / 2, ylim[1] + (d1 - d2) / 2))
        else:
            plt.xlim((xlim[0] + (d1 - d2) / 2, xlim[1] - (d1 - d2) / 2))
    else:
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            ax.set_ylim((ylim[0] - (d1 - d2) / 2, ylim[1] + (d1 - d2) / 2))
        else:
            ax.set_xlim((xlim[0] + (d1 - d2) / 2, xlim[1] - (d1 - d2) / 2))


In [32]:
# 2.1: f, grad
def func(x):
    return np.e ** (x[0] + 3 * x[1]) + np.e ** (x[0] - 3 * x[1]) + np.e ** (-x[0])


def f_grad(x):
    return np.array([np.e ** (x[0] + 3 * x[1]) + np.e ** (x[0] - 3 * x[1]) - np.e ** (-x[0]),
                     3 * (np.e ** (x[0] + 3 * x[1]) - np.e ** (x[0] - 3 * x[1]))])


min_point = [-np.log(2) / 2, 0]

In [33]:
# 2.2: estimation of Lipschitz constant
def get_lipschitz_grid(x_range, y_range):
    L = 0
    delta = 0.1
    x = np.arange(x_range[0], x_range[1], delta)
    y = np.arange(y_range[0], y_range[1], delta)
    X, Y = np.meshgrid(x, y)
    ZX = np.zeros_like(X)
    ZY = np.zeros_like(X)
    # print(X.shape, Y.shape)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            grad = f_grad([X[i][j], Y[i][j]])
            ZX[i][j] = grad[0]
            ZY[i][j] = grad[1]
    # print(range(X.shape[0]))
    # print(range(X.shape[1]))
    for i1 in range(X.shape[0]):
        for j1 in range(X.shape[1]):
            for i2 in range(X.shape[0]):
                for j2 in range(X.shape[1]):
                    x1 = X[i1][j1]
                    y1 = Y[i1][j1]
                    x2 = X[i2][j2]
                    y2 = Y[i2][j2]  
                    # print("Point1: ", x1, y1)
                    # print("Point2: ", x2, y2)
                    norm_points = np.linalg.norm(np.array([x1, y1]) - np.array([x2, y2]))
                    norm_f = np.linalg.norm(np.array([ZX[i1][j1], ZY[i1][j1]]) - np.array([ZX[i2][j2], ZY[i2][j2]]))
                    # print("Norm point: ", norm_points)
                    # print("Norm f: ", norm_f)
                    if norm_points == 0:
                        continue
                    new_L = norm_f / norm_points
                    # print("New L: ", new_L)
                    L = max(L, new_L)
    return L

In [34]:
# 2.2: estimation of Lipschitz constant
def get_lipschitz_random(x_range, y_range):
    L = 0
    iterations_number = 100000
    for i in range(iterations_number):
        rand = np.random.random([4])
        x1 = x_range[0] + (x_range[1] - x_range[0]) * rand[0]
        y1 = y_range[0] + (y_range[1] - y_range[0]) * rand[1]
        x2 = x_range[0] + (x_range[1] - x_range[0]) * rand[2]
        y2 = y_range[0] + (y_range[1] - y_range[0]) * rand[3]
        # print("Point1: ", x1, y1)
        # print("Point2: ", x2, y2)
        norm_points = np.linalg.norm(np.array([x1, y1]) - np.array([x2, y2]))
        norm_f = np.linalg.norm(np.array(f_grad([x1, y1])) - np.array(f_grad([x2, y2])))
        # print("Norm point: ", norm_points)
        # print("Norm f: ", norm_f)
        if norm_points == 0:
            continue
        new_L = norm_f / norm_points
        # print("New L: ", new_L)
        L = max(L, new_L)
    return L

In [35]:
# 2.2: estimation of Lipschitz constant
def get_lipschitz(x_range, y_range):
    return max(get_lipschitz_grid(x_range, y_range), get_lipschitz_random(x_range, y_range))

In [36]:
# 2.2: estimation of Lipschitz constant
L_constant = get_lipschitz_grid([-1, 1], [-1, 1])
print("Grid: ", L_constant)

Grid:  405.98947162419614


In [37]:
# 2.2: estimation of Lipschitz constant
L_constant = get_lipschitz_random([-1, 1], [-1, 1])
print("Random:", L_constant)

Random: 447.50240455558475


In [38]:
# 2.2: estimation of Lipschitz constant
L_constant = get_lipschitz([-1, 1], [-1, 1])
print("Lipschitz constant approximately equals ", L_constant)

Lipschitz constant approximately equals  405.98947162419614


In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML


def animate_trajectory(trajectory):
    fig, ax = plt.subplots()
    n = len(trajectory)

    def step(t):
        ax.cla()
        ax.plot([min_point[0]], [min_point[1]], 'o', color='green')
        # Level contours
        delta = 0.025
        x = np.arange(-2, 4, delta)
        y = np.arange(-3, 3, delta)
        X, Y = np.meshgrid(x, y)
        Z = np.zeros_like(X)
        # print(X.shape, Y.shape)
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                Z[i][j] = func([X[i][j], Y[i][j]])
        CS = ax.contour(X, Y, Z, [1, 6, 20], colors=['blue', 'purple', 'red'])

        ax.plot([u[0] for u in trajectory[:t]], [u[1] for u in trajectory[:t]], color='black')
        ax.plot([u[0] for u in trajectory[:t]], [u[1] for u in trajectory[:t]], 'o', color='black')
        
        fix_scaling(ax)
        ax.axis('off')

    return FuncAnimation(fig, step, frames=range(n), interval=600)

In [ ]:
alpha = 1 / (1 + 1)
beta = (np.sqrt(1) - np.sqrt(2)) / (np.sqrt(1) + np.sqrt(2))
trajectory_nesterov = []
x_start = np.array([2, 2.8])
trajectory_nesterov.append(x_start.copy())
cur_x = x_start.copy()
cur_y = x_start.copy()
iterations_number = 25
for i in range(iterations_number):
    t = cur_x
    cur_x = cur_y - alpha * f_grad(cur_y)
    cur_y = cur_x + beta * (cur_x - t)
    trajectory_nesterov.append(cur_x.copy())
    
base_animation = animate_trajectory(trajectory_nesterov)
HTML(base_animation.to_html5_video())